# 🔍 Azure Document Extraction Pipeline
## Using Responses API with Structured Outputs

### Architecture
```
📄 Document (PDF/Image)
       │
       ▼
┌──────────────────────────┐
│  Mistral Document AI     │  ← OCR + Text Extraction
│  (mistral-ocr-2503)      │
└──────────────────────────┘
       │
       ▼ OCR Text + Original Image
┌──────────────────────────┐
│  GPT-5.1                 │  ← Structured Extraction (Responses API)
│  (Responses API +        │     with strict JSON Schema
│   Structured Outputs)    │
└──────────────────────────┘
       │
       ▼ Extraction Results
┌──────────────────────────┐
│  Azure AI Evaluation SDK │  ← Quality Evaluation
└──────────────────────────┘
```

## 1️⃣ Install Dependencies

Run the cell below to install required packages. Key libraries:
- `openai` - Azure OpenAI client for GPT models
- `azure-ai-evaluation` - Quality metrics (Groundedness, Relevance, Coherence)
- `azure-identity` - Entra ID authentication
- `pydantic` - Schema definition for Structured Outputs

In [1]:
#!pip install -q --upgrade openai azure-ai-evaluation azure-identity python-dotenv pydantic requests

## 2️⃣ Configuration

Create a `.env` file in the same directory with the following variables:

```env
# Mistral Document AI
MISTRAL_ENDPOINT=https://<your-resource>.services.ai.azure.com
MISTRAL_API_KEY=<your-api-key>
MISTRAL_MODEL=mistral-ocr-2503

# Azure OpenAI (Responses API)
# Found in: Foundry > Models + endpoints > Your deployment
AZURE_OPENAI_ENDPOINT=https://<your-resource>.cognitiveservices.azure.com
AZURE_OPENAI_API_KEY=<your-api-key>
AZURE_OPENAI_DEPLOYMENT=gpt-5.1

# Evaluation Model
EVAL_MODEL_DEPLOYMENT=gpt-4o
```

In [2]:
# Add src/ to path for imports
import sys
sys.path.insert(0, 'src')

# Import from our modules
from config import Config
from utils import get_file_info, encode_file_to_base64, is_pdf
from ocr import MistralOCR, create_ocr_client
from extractor import StructuredExtractor, create_extractor
from evaluator import QualityEvaluator, ExtractionValidator, create_evaluator
from confidence import ConfidenceCalculator, create_confidence_calculator  # NEW: OCR vs Extraction comparison
from schemas import DocumentExtraction, EXTRACTION_SCHEMA, get_strict_schema
from pipeline import DocumentPipeline

# Standard imports for notebook usage
import json
from pathlib import Path
from typing import Dict, Any, Optional

print("✅ All modules imported successfully")
print("   - Config, Utils, OCR, Extractor, Evaluator, Confidence, Schemas, Pipeline")

✅ All modules imported successfully
   - Config, Utils, OCR, Extractor, Evaluator, Confidence, Schemas, Pipeline


In [3]:
# =============================================================================
# Configuration - Load from .env file
# =============================================================================

# Load configuration from environment variables
config = Config.from_env()

# Display configuration status
status = config.show_status()

# Verify all required services are configured
if not status["mistral_configured"]:
    print("\n⚠️ Warning: Mistral OCR not configured - check MISTRAL_ENDPOINT and MISTRAL_API_KEY")
if not status["aoai_configured"]:
    print("\n⚠️ Warning: Azure OpenAI not configured - check AZURE_OPENAI_ENDPOINT")

print(f"\n📋 Extraction mode: {config.extraction_mode}")

Configuration Status
✅ Mistral Document AI
   Endpoint: https://felat-mgtjiioy-francecentral.cognitiveserv...
   Model: mistral-document-ai-2505-2

✅ Azure OpenAI (Responses API)
   Endpoint: https://doc-processing-mistral-gpt-eval.cognitives...
   Base URL: https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com/openai/v1/
   Deployment: gpt-5.1

✅ Evaluation: Configured
   Deployment: gpt-4o

📋 Extraction mode: hybrid


## 3️⃣ Define Your Extraction Schema

We use **Pydantic models** to define what data we want to extract. Azure OpenAI's **Structured Outputs** feature guarantees the model returns valid JSON matching our schema exactly.

The schema is converted to "strict mode" which requires `additionalProperties: false` and all fields in `required`.

In [4]:
# =============================================================================
# Extraction Schema - Already imported from src/schemas.py
# =============================================================================

# The schema is imported from src/schemas.py
# You can view the Pydantic models: DocumentExtraction, LineItem, MonetaryAmount, PersonInfo

print("📋 Extraction Schema (Strict Mode for Azure OpenAI):")
print(f"   Schema name: DocumentExtraction")
print(f"   Fields: {list(EXTRACTION_SCHEMA.get('properties', {}).keys())}")
print("\n   💡 To see full schema: print(json.dumps(EXTRACTION_SCHEMA, indent=2))")

📋 Extraction Schema (Strict Mode for Azure OpenAI):
   Schema name: DocumentExtraction
   Fields: ['document_type', 'document_number', 'document_date', 'supplier', 'customer', 'total_amount', 'line_items']

   💡 To see full schema: print(json.dumps(EXTRACTION_SCHEMA, indent=2))


## 4️⃣ Document Utilities

Helper functions to handle document files: encode to base64, detect MIME types, and check file properties.

In [5]:
# =============================================================================
# Document Utilities - Already imported from src/utils.py
# =============================================================================

# Functions available:
# - encode_file_to_base64(file_path) -> (base64_string, mime_type)
# - get_file_info(file_path) -> dict with name, extension, size
# - is_pdf(file_path) -> bool

# Test with a sample file
test_file = "invoice.png"
if Path(test_file).exists():
    info = get_file_info(test_file)
    print(f"✅ Document utilities ready")
    print(f"   Test file: {info['name']} ({info['size_mb']} MB)")
else:
    print("✅ Document utilities ready")
    print("   ℹ️ Place a document (PDF/image) to test")

✅ Document utilities ready
   Test file: invoice.png (0.353 MB)


## 5️⃣ Step 1: OCR with Mistral Document AI

**Mistral Document AI** is a specialized OCR model that extracts text from documents (PDFs, images) and returns it in **Markdown format**.

This is powerful because:
- Tables are preserved as Markdown tables
- Headers and structure are maintained
- The output is clean and LLM-ready

The OCR text becomes the **context** for our extraction model and serves as **ground truth** for confidence calculation.

In [6]:
# =============================================================================
# OCR Client - Using src/ocr.py
# =============================================================================

# Create OCR client from configuration
ocr_client = create_ocr_client(config)

print(f"✅ Mistral OCR client initialized")
print(f"   Model: {config.mistral_model}")
print(f"   Endpoint: {config.mistral_endpoint[:50]}...")

✅ Mistral OCR client initialized
   Model: mistral-document-ai-2505-2
   Endpoint: https://felat-mgtjiioy-francecentral.cognitiveserv...


## 6️⃣ Step 2: Structured Extraction with GPT-5.1

GPT-5.1 extracts structured data from the OCR text using our Pydantic schema.

**Two extraction modes available:**
- `text_only` - Uses only OCR text (faster, cheaper)
- `hybrid` - Sends both OCR text AND the original image (more accurate for complex layouts)

The model returns JSON that exactly matches our schema - no parsing errors possible!

In [7]:
# =============================================================================
# Structured Extractor - Using src/extractor.py
# =============================================================================

# Create extractor from configuration (uses Entra ID auth)
extractor = create_extractor(config)

print(f"✅ StructuredExtractor initialized with Entra ID")
print(f"   Deployment: {config.aoai_deployment}")
print(f"   Extraction mode: {config.extraction_mode}")
print(f"   Available methods:")
print(f"     - extract(text, schema)           # Text only")
print(f"     - extract_hybrid(ocr_text, image) # OCR + Image (recommended)")

✅ StructuredExtractor initialized with Entra ID
   Deployment: gpt-5.1
   Extraction mode: hybrid
   Available methods:
     - extract(text, schema)           # Text only
     - extract_hybrid(ocr_text, image) # OCR + Image (recommended)


## 7️⃣ Step 3: Quality Evaluation

This is the **core of our pipeline**. We evaluate extraction quality using two complementary approaches:

---

### 🎯 Approach 1: Confidence Score (OCR vs Extraction) — No LLM!

We **compare extracted values against the original OCR text** using pure text matching (no AI).

#### How it works (Ctrl+F approach)

```
GPT extracts: { "supplier_address": "123 Main Street" }

                    │
                    ▼

    Is "123 Main Street" somewhere in the OCR text?
    (Like doing Ctrl+F in a Word document)

                    │
        ┌───────────┴───────────┐
        │                       │
        ▼                       ▼
    FOUND                   NOT FOUND
    Confidence = 1.0 ✅      Confidence = 0.0 ❌
                            (Hallucination!)
```

#### Fuzzy Matching (tolerates typos)

What if there's a small difference? We use **fuzzy matching** (`difflib.SequenceMatcher`):

```python
from difflib import SequenceMatcher

# Compare two strings
ratio = SequenceMatcher(None, "Chan Accounting", "Chan Acounting").ratio()
# ratio = 0.96 (96% similar)

if ratio >= 0.7:  # Our threshold
    confidence = ratio  # ⚠️ Fuzzy match (keeps actual ratio!)
else:
    confidence = 0.0    # ❌ Not found
```

| OCR Text | GPT Extracted | Similarity | Result |
|----------|---------------|------------|--------|
| "Chan Accounting" | "Chan Accounting" | 100% | ✅ 1.0 (exact) |
| "Chan Accounting" | "Chan Acounting" | 96% | ⚠️ 0.96 (fuzzy) |
| "Chan Accounting" | "Chan Acount" | 85% | ⚠️ 0.85 (fuzzy) |
| "Chan Accounting" | "John Smith" | 20% | ❌ 0.0 (hallucination) |

> ⚠️ **Important**: The 70% threshold determines if it's a match or not, but the **confidence score = the actual similarity ratio**. So 70% match → score of 0.70, not 1.0. This penalizes fuzzy matches proportionally.

> 💡 **Key insight**: This is **deterministic** (same input = same output), **free** (no API calls), and **fast**. But it only checks if the text exists, not if it's semantically correct.

---

### 🎯 Approach 2: Azure AI Evaluation SDK (LLM-as-a-Judge)

These are **AI-assisted evaluators** that use **GPT-4o as a judge** to assess quality:

| Metric | What it measures | Why it matters |
|--------|------------------|----------------|
| **Groundedness** | Is the extraction supported by the source text? | Detects hallucinations semantically |
| **Relevance** | Does the extraction answer the query? | Ensures we extracted the RIGHT fields |
| **Coherence** | Is the output logically structured? | Validates JSON quality and readability |

#### How LLM-as-a-Judge works

The SDK sends to the judge model (GPT-4o):
1. **Prompt template** with metric definition and scoring rubric (Likert scale 1-5)
2. **Context** = OCR text (grounding source)
3. **Response** = Extracted JSON
4. **Query** = Our extraction instruction

The judge model returns:
- A **score** (1-5) for each metric
- A **reason** explaining why (useful for debugging)

```
┌─────────────────────────────────────────────────────────┐
│                    GPT-4o (Judge)                       │
├─────────────────────────────────────────────────────────┤
│  Input:                                                 │
│  - Prompt: "Evaluate groundedness on scale 1-5..."      │
│  - Context: "Invoice #0324, Date: 2024-01-15..."        │
│  - Response: {"document_number": "0324", ...}           │
│  - Query: "Extract document information"                │
├─────────────────────────────────────────────────────────┤
│  Output:                                                │
│  - groundedness: 5                                      │
│  - groundedness_reason: "All extracted values are       │
│    directly supported by the source text..."            │
└─────────────────────────────────────────────────────────┘
```

---

### 🔄 How Both Approaches Work Together

The two approaches are **independent and complementary**. They run in parallel and answer different questions:

```
┌─────────────────────────────────────────────────────────────────┐
│                      PIPELINE FLOW                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  OCR Text ─────────┬──────────────────────────────────────────▶ │
│                    │                                            │
│  Extracted JSON ───┼──────────────────────────────────────────▶ │
│                    │                                            │
│                    ▼                                            │
│         ┌─────────────────────┐    ┌─────────────────────────┐  │
│         │  Approach 1:        │    │  Approach 2:            │  │
│         │  Fuzzy Matching     │    │  LLM-as-a-Judge         │  │
│         │  (per field)        │    │  (whole extraction)     │  │
│         └──────────┬──────────┘    └───────────┬─────────────┘  │
│                    │                           │                │
│                    ▼                           ▼                │
│         confidence: 0.92           groundedness: 5              │
│         (field-level scores)       relevance: 5                 │
│                                    coherence: 4                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

#### What each metric tells you

| Question | Fuzzy | Groundedness | Relevance | Coherence |
|----------|-------|--------------|-----------|-----------|
| "Is the text physically present in OCR?" | ✅ | ❌ | ❌ | ❌ |
| "Is the meaning correct (even if reformatted)?" | ❌ | ✅ | ❌ | ❌ |
| "Did we extract the RIGHT field?" | ❌ | ❌ | ✅ | ❌ |
| "Is the JSON well-structured?" | ❌ | ❌ | ❌ | ✅ |

---

### 📊 Interpretation Matrix: How to Read Combined Results

Use this matrix to diagnose extraction quality:

| Fuzzy | Ground. | Relev. | Coher. | Diagnosis | Action |
|-------|---------|--------|--------|-----------|--------|
| **1.0** | **5** | **5** | **5** | ✅ **Perfect** — Exact text, correct meaning, right field, clean JSON | Trust the result |
| **0.7-0.99** | **5** | **5** | **5** | ⚠️ **Minor OCR issue** — Small typo but everything else correct | Acceptable |
| **0.0** | **5** | **5** | **5** | 🔄 **Reformatted** — Different format but semantically correct | Acceptable |
| **1.0** | **5** | **1-2** | **5** | 🎯 **Wrong field** — Text exists but assigned to wrong field | Review schema/prompt |
| **1.0** | **5** | **5** | **1-2** | 📝 **Bad JSON** — Values OK but structure issues | Check schema |
| **0.0** | **1-2** | **?** | **?** | ❌ **Hallucination** — GPT invented this value | Reject |

#### Concrete example

```
Document OCR: "Invoice Date: 15/01/2024, Delivery Date: 20/01/2024"

Extraction: { "invoice_date": "20/01/2024" }  ← ERROR: wrong date!

Scores:
┌────────────────┬───────┬─────────────────────────────────────────┐
│ Metric         │ Score │ Why                                     │
├────────────────┼───────┼─────────────────────────────────────────┤
│ Fuzzy          │ 1.0 ✅│ "20/01/2024" exists in OCR text         │
│ Groundedness   │ 5 ✅  │ Value comes from the document           │
│ Relevance      │ 1 ❌  │ Wrong date! (delivery vs invoice)       │
│ Coherence      │ 5 ✅  │ JSON is well-formed                     │
└────────────────┴───────┴─────────────────────────────────────────┘

Diagnosis: Relevance catches the error that Fuzzy + Groundedness missed!
```

#### Another example: Date reformatting

```
Document OCR: "Date: 15/01/2024"

Extraction: { "invoice_date": "January 15, 2024" }  ← Same date, different format

Scores:
┌────────────────┬───────┬─────────────────────────────────────────┐
│ Metric         │ Score │ Why                                     │
├────────────────┼───────┼─────────────────────────────────────────┤
│ Fuzzy          │ 0.0 ❌│ "January 15, 2024" ≠ "15/01/2024"       │
│ Groundedness   │ 5 ✅  │ Same date, just reformatted             │
│ Relevance      │ 5 ✅  │ Correct field                           │
│ Coherence      │ 5 ✅  │ JSON is well-formed                     │
└────────────────┴───────┴─────────────────────────────────────────┘

Diagnosis: Fuzzy fails but Groundedness confirms it's correct → Acceptable!
```

> 💡 **Key takeaway**: No single metric tells the whole story. Use them together:
> - **Fuzzy** = Fast/free sanity check
> - **Groundedness** = Semantic correctness
> - **Relevance** = Right field assignment
> - **Coherence** = Output quality

---

### 🔄 Comparison: Fuzzy Matching vs LLM-as-a-Judge

| Aspect | Fuzzy Matching (Approach 1) | LLM-as-a-Judge (Approach 2) |
|--------|-----------------------------|-----------------------------|
| **Cost** | Free (pure Python) | Paid (API call to GPT-4o) |
| **Speed** | Instant | ~1-2 seconds per evaluation |
| **Intelligence** | Text matching only | Semantic understanding |
| **Determinism** | Same input = same output | May vary slightly |
| **Use case** | Quick hallucination check | Detailed quality assessment |

> ⚠️ **Important**: LLM-as-a-Judge requires a **separate model deployment** (GPT-4o). Configure `EVAL_MODEL_DEPLOYMENT=gpt-4o` in your `.env` file.

> 📚 **Documentation**: [RAG Evaluators](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/rag-evaluators) | [General Purpose Evaluators](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-evaluators/general-purpose-evaluators)

In [ ]:
# =============================================================================
# Extraction Validator & Confidence Calculator - Using src/evaluator.py & src/confidence.py
# =============================================================================

# Create validator instance (dates, amounts)
validator = ExtractionValidator()

# Create confidence calculator (OCR vs Extraction comparison)
# fuzzy_threshold=0.7 means: 70% text similarity = considered a match
# Lower = more permissive (tolerates OCR errors), Higher = stricter
confidence_calc = create_confidence_calculator(fuzzy_threshold=0.7)

print("✅ Extraction validator initialized")
print("   Validates: dates, amounts")
print("\n✅ Confidence Calculator initialized")
print("   Method: Text matching (Ctrl+F) + Fuzzy matching (difflib)")
print("   Threshold: 0.7 (70% similarity = match)")
print("   💡 No LLM used - pure Python, deterministic, free!")

✅ Extraction validator initialized
   Validates: dates, amounts

✅ Confidence Calculator initialized
   Method: OCR vs Extraction text comparison
   No more self-evaluation from GPT!


In [9]:
# =============================================================================
# Quality Evaluator - Using src/evaluator.py
# =============================================================================

# Create evaluator from configuration (uses GPT-4o as judge)
if config.eval_deployment:
    evaluator = create_evaluator(config)
    print(f"✅ QualityEvaluator initialized")
    print(f"   Judge model: {config.eval_deployment}")
    print(f"   Metrics: Groundedness, Relevance, Coherence")
else:
    evaluator = None
    print("⚠️ Evaluator not configured (EVAL_MODEL_DEPLOYMENT not set)")
    print("   Set EVAL_MODEL_DEPLOYMENT=gpt-4o in .env to enable evaluation")

✅ QualityEvaluator initialized
   Judge model: gpt-4o
   Metrics: Groundedness, Relevance, Coherence


## 8️⃣ Complete Pipeline

The `DocumentPipeline` class orchestrates all components: OCR → Extraction → Confidence → Evaluation.

In [10]:
# =============================================================================
# Complete Pipeline - Using src/pipeline.py
# =============================================================================

# Create pipeline from configuration
pipeline = DocumentPipeline(config)

print("\n📝 Usage:")
print('   results = pipeline.process("document.pdf")')
print('   results = pipeline.process("document.pdf", run_evaluation=True)')
print('   pipeline.display_results(results)')

🔧 Initializing pipeline components...
   ✅ OCR client ready (model: mistral-document-ai-2505-2)
   ✅ Extractor ready (model: gpt-5.1)
   ✅ Evaluator ready (model: gpt-4o)
   ✅ Confidence calculator ready (OCR vs Extraction comparison)
✅ Pipeline ready


📝 Usage:
   results = pipeline.process("document.pdf")
   results = pipeline.process("document.pdf", run_evaluation=True)
   pipeline.display_results(results)


## 9️⃣ Example Usage

Let's process a real document and see all components in action.

In [ ]:
# =============================================================================
# Display the document we're about to process
# =============================================================================
from IPython.display import Image, display

document_path = "invoice.png"

if Path(document_path).exists():
    print(f"📄 Document: {document_path}")
    print(f"   Size: {get_file_info(document_path)['size_mb']} MB")
    print("\n👇 Preview:")
    display(Image(filename=document_path, width=600))
else:
    print(f"⚠️ Document not found: {document_path}")
    print("   Place an invoice.png file in the project root to test.")

In [11]:
# =============================================================================
# Process a document
# =============================================================================

# Process the sample invoice
results = pipeline.process(
    file_path="invoice.png",
    run_evaluation=True,
    verbose=True
)

📄 Processing: invoice.png
   Size: 0.353 MB

🔄 Step 1: OCR with Mistral Document AI
   ✅ OCR complete (1 pages)

🔄 Step 2: Hybrid extraction with GPT-5.1 (OCR + Image)
   ✅ Extraction complete (mode: hybrid)
   📊 Confidence (OCR vs Extraction): 87.0%

🔄 Step 3: Quality evaluation
   ✅ Evaluation complete (score: 77.8%)

✅ Processing complete


In [12]:
# =============================================================================
# Display Results
# =============================================================================

pipeline.display_results(results)


📊 EXTRACTION RESULTS

Document Type: invoice
Document Number: 0324
Document Date: None
Total Amount: 500.0 USD

📊 Confidence Score: 87.0% (method: ocr_extraction_comparison)
   ✅ High confidence fields: 5
   ⚠️  Medium confidence: 0
   ❌ Low confidence: 1

Line Items (5):
   1. Sed ut perspiciatis unde omnis iste lore... → 100.0
   2. Adipisc ing elit, sed do eius mod tempor... → 100.0
   3. Nemo enim ipsam volupt atem quia volupta... → 100.0
   4. Sed ut perspiciatis unde omnis iste lore... → 100.0
   5. Adipisc ing elit, sed do eius mod tempor... → 100.0

📈 EVALUATION RESULTS
   ⚠️ Groundedness: 2.0/5
   ✅ Relevance: 5.0/5
   ✅ Coherence: 4.0/5

Validation: 2/2 checks passed

⚠️ OVERALL SCORE: 77.8%


In [ ]:
# Check extraction mode used
metadata = results.get("extraction", {}).get("_metadata", {})
print("🔍 Extraction Metadata:")
print(f"   Mode: {metadata.get('mode', 'NOT DEFINED - probably text_only')}")
print(f"   Input tokens: {metadata.get('usage', {}).get('input_tokens', 'N/A')}")
print(f"   Output tokens: {metadata.get('usage', {}).get('output_tokens', 'N/A')}")

# If mode is not "hybrid_ocr_vision", the image was not sent
if metadata.get('mode') == 'hybrid_ocr_vision':
    print("\n✅ Image was sent along with OCR text")
else:
    print("\n⚠️ Text-only mode - only OCR text was sent")

🔍 Métadonnées de l'extraction:
   Mode: hybrid_ocr_vision
   Input tokens: 1608
   Output tokens: 260

✅ L'image a bien été envoyée avec le texte OCR


In [ ]:
# =============================================================================
# Confidence Score Details (OCR vs Extraction)
# =============================================================================

confidence_report = results.get("confidence", {})

print("🔍 CONFIDENCE SCORE DETAILS")
print("=" * 60)
print(f"Method: {confidence_report.get('method', 'N/A')}")
print(f"Overall score: {confidence_report.get('overall_confidence', 0):.1%}")
print()

# Display field breakdown
print("📋 Field breakdown:")
for field_name, field_data in confidence_report.get("fields", {}).items():
    conf = field_data.get("confidence", 0)
    note = field_data.get("note", "")
    extracted = field_data.get("extracted", "N/A")
    icon = "✅" if conf >= 0.9 else "⚠️" if conf >= 0.5 else "❌"
    print(f"   {icon} {field_name}: {conf:.1%} ({note})")
    print(f"      Extracted value: {extracted}")

print()
summary = confidence_report.get("summary", {})
print(f"📊 Summary: {summary.get('high_confidence', 0)} high | {summary.get('medium_confidence', 0)} medium | {summary.get('low_confidence', 0)} low")

🔍 DÉTAIL DU SCORE DE CONFIANCE
Méthode: ocr_extraction_comparison
Score global: 87.0%

📋 Détail par champ:
   ✅ document_number: 100.0% (exact match)
      Valeur extraite: 0324
   ✅ document_date: 100.0% (Field is null)
      Valeur extraite: None
   ❌ supplier_name: 0.0% (not found in OCR)
      Valeur extraite: Chan Accounting
   ✅ customer_name: 100.0% (exact match)
      Valeur extraite: Paul J Gordon
   ✅ total_amount: 100.0% (exact match)
      Valeur extraite: 500.00
   ✅ line_items: 100.0% (average across 5 items)
      Valeur extraite: 5 items

📊 Résumé: 5 élevés | 0 moyens | 1 faibles


In [ ]:
# Comparison test: text_only vs hybrid mode
print("🔬 Extraction Mode Comparison\n")

# Save current mode
original_mode = config.extraction_mode

# Test in text_only mode
config.extraction_mode = "text_only"
pipeline_text = DocumentPipeline(config)
results_text = pipeline_text.process("invoice.png", run_evaluation=False, verbose=False)

# Restore hybrid mode
config.extraction_mode = "hybrid"

print("📊 Results:")
print(f"\n{'Mode':<15} {'Input Tokens':<15} {'Confidence (OCR vs Extract)':<25}")
print("-" * 55)

meta_hybrid = results.get("extraction", {}).get("_metadata", {})
meta_text = results_text.get("extraction", {}).get("_metadata", {})

conf_hybrid = results.get("confidence", {}).get("overall_confidence", "N/A")
conf_text = results_text.get("confidence", {}).get("overall_confidence", "N/A")

print(f"{'hybrid':<15} {meta_hybrid.get('usage', {}).get('input_tokens', 'N/A'):<15} {conf_hybrid}")
print(f"{'text_only':<15} {meta_text.get('usage', {}).get('input_tokens', 'N/A'):<15} {conf_text}")

🔬 Comparaison des modes d'extraction

🔧 Initializing pipeline components...
   ✅ OCR client ready (model: mistral-document-ai-2505-2)
   ✅ Extractor ready (model: gpt-5.1)
   ✅ Evaluator ready (model: gpt-4o)
   ✅ Confidence calculator ready (OCR vs Extraction comparison)
✅ Pipeline ready

📊 Résultats:

Mode            Input Tokens    Confidence (OCR vs Extract)
-------------------------------------------------------
hybrid          1608            0.87
text_only       932             1.0


In [16]:
# =============================================================================
# Export Results
# =============================================================================

pipeline.export_results(results, "output/extraction_results.json")

✅ Results exported to: output/extraction_results.json


## 🎯 Summary

This notebook provides a document extraction pipeline using:

1. **Mistral Document AI** for OCR
2. **GPT-5.1 via Responses API** with strict JSON Schema validation
3. **Confidence Calculation** by comparing OCR text with extracted values (no self-evaluation from GPT!)
4. **Azure AI Evaluation SDK** for semantic quality assessment (Groundedness, Relevance, Coherence)

### Confidence Score Method
The confidence score is now calculated by **comparing the extracted values against the original OCR text**:
- **Perfect match** (value found exactly in OCR) → 1.0
- **Fuzzy match** (similar text found) → 0.5-0.99
- **Not found** → 0.0

This is more reliable than asking GPT to self-evaluate its confidence.

### To customize:
1. Modify the schema in Section 3 for your document type
2. Update validators in Section 7 for your rules
3. Adjust `fuzzy_threshold` in ConfidenceCalculator (default: 0.8)
4. Use `custom_instructions` when calling `process_document()`